In [1]:
import tensorflow as tf

/Users/yannick.habecker/projects/personal/printer-anomaly-detection/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/yannick.habecker/projects/personal/printer-anomaly-detection/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/yannick.habecker/projects/personal/printer-anomaly-detection/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/yannick.habecker/projects/personal/printer-anomaly-detection/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/yannick.hab

In [136]:
from typing import Tuple

class CAE(tf.keras.Model):
    """Convolutional autoencoder."""

    def __init__(self, latent_dim: int = 200, input_shape: Tuple[int, int, int] = (256, 256, 1)):
      super(CAE, self).__init__()
      self.latent_dim = latent_dim
      self._input_shape = input_shape

      self.encode = tf.keras.Sequential(
          [
              tf.keras.layers.InputLayer(input_shape=self._input_shape),
              tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(latent_dim),
          ]
      )

      self.decode = tf.keras.Sequential(
          [
              tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
              tf.keras.layers.Dense(units=4096, activation='relu'),
              tf.keras.layers.Reshape(target_shape=(8, 8, 64)),
              tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=2, padding='same', activation='sigmoid'),
          ]
      )

    def call(self, inputs, training=None, mask=None):
      encoded = self.encode(inputs)
      return self.decode(encoded)


model = CAE()
model.build(input_shape=(None, 256, 256, 1))
model.summary()

Model: "cae_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_162 (Sequential)  (None, 200)              757896    
                                                                 
 sequential_163 (Sequential)  (None, 256, 256, 3)      954307    
                                                                 
Total params: 1,712,203
Trainable params: 1,711,307
Non-trainable params: 896
_________________________________________________________________
